In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

class Flatten(torch.nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
num_of_feature = 11

In [2]:
# model = torch.nn.Sequential(
#     torch.nn.Linear(13, 64),
#     torch.nn.LeakyReLU(),
#     torch.nn.Linear(64, 32),
#     torch.nn.LeakyReLU(),
#     torch.nn.Linear(32, 2),
# )

model = torch.nn.Sequential(
    torch.nn.Conv1d(1, 64, 3, 1), #11
    torch.nn.Tanh(),
    torch.nn.Conv1d(64, 128, 3, 1), #9
    torch.nn.Tanh(),
    Flatten(),
    torch.nn.Linear(128*(num_of_feature-4), 2)
#     torch.nn.Linear(128, 2)
#     torch.nn.Linear(576, 2)
    )

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [3]:
def train(batch, epoch, dataset):
    x_train = np.load('data/x_train_11.npy') #420
    x_val = np.load('data/x_val_11.npy')     #91
    x_test = np.load('data/x_test_11.npy')   #90
    y_train = np.load('data/y_train_' + dataset + '_11.npy')
    y_val = np.load('data/y_val_' + dataset + '_11.npy')
    y_test = np.load('data/y_test_' + dataset + '_11.npy')
    x_train = np.expand_dims(x_train, axis=1)
    x_val = np.expand_dims(x_val, axis=1)
    x_test = np.expand_dims(x_test, axis=1)
    data_pred = np.load('data/for_pred_11.npy')
    data_pred = np.expand_dims(data_pred, axis=1)
    indicies = np.arange(x_train.shape[0])
    epoch_loss = []
    epoch_accuracy = []
    val_accuracy = []
    test_accuracy = []
    IM_accuracy = []
    data_pred_labels = []
    for e in range(epoch):
        losses = []
        accuracies = []
        for i in range(0, int(np.ceil(x_train.shape[0]/batch))):
            start_idx = (i * batch) % x_train.shape[0]
            idx = indicies[start_idx: start_idx + batch]
            x_in = torch.from_numpy(x_train[idx]).type(torch.float32)
            y_in = torch.from_numpy(y_train[idx]).type(torch.long)
            y_score = model(x_in)
            predict = torch.argmax(y_score, 1)
            difference = predict == y_in
            accuracy = torch.mean(difference.type(torch.float32))
            loss = torch.nn.CrossEntropyLoss()
            loss = loss(y_score, y_in)
#             print('loss: ', loss.item(),' accuracy: ', accuracy.item())
            losses.append(loss.item())
            accuracies.append(accuracy.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('epoch ', e, ' with loss: %.4f' %np.mean(losses), ' and accuracy: %.4f' %np.mean(accuracies))
        epoch_loss.append(np.mean(losses))
        epoch_accuracy.append(np.mean(accuracies))
        
        x_val_in = torch.from_numpy(x_val).type(torch.float32)
        y_val_in = torch.from_numpy(y_val).type(torch.long)
        y_val_score = model(x_val_in)
        val_predict = torch.argmax(y_val_score, 1) == y_val_in
        val_acc = torch.mean(val_predict.type(torch.float32))
        val_accuracy.append(val_acc.item())
        
        x_test_in = torch.from_numpy(x_test).type(torch.float32)
        y_test_in = torch.from_numpy(y_test).type(torch.long)
        y_test_score = model(x_test_in)
        test_predict = torch.argmax(y_test_score, 1) == y_test_in
        test_acc = torch.mean(test_predict.type(torch.float32))
        test_accuracy.append(test_acc.item())
        
        data_pred_in = torch.from_numpy(data_pred).type(torch.float32)
        data_pred_score = model(data_pred_in)
        data_pred_label = torch.argmax(data_pred_score, 1)
        data_pred_labels.append(data_pred_label)
        
        print('epoch ', e, ' with val_accuracy: %.4f' %val_acc.item(), ' and test_accuracy: %.4f' %test_acc.item())
    
    print('Highest val accuracy: %.4f' %np.max(val_accuracy), ' and test accuracy: %.4f' %np.max(test_accuracy))
    plt.xlabel('Loss')
    plt.plot(epoch_loss[10:])
    plt.grid(True)
    plt.show()
    plt.xlabel('Accuracy')
    plt.plot(epoch_accuracy)
    plt.grid(True)
    plt.show()
    return data_pred_labels

In [4]:
data_pred_labels = train(20, 200, 'SS')

epoch  0  with loss: 0.2487  and accuracy: 0.8952
epoch  0  with val_accuracy: 0.9780  and test_accuracy: 0.9333
epoch  1  with loss: 0.1547  and accuracy: 0.9619
epoch  1  with val_accuracy: 0.9780  and test_accuracy: 0.9444
epoch  2  with loss: 0.1662  and accuracy: 0.9524
epoch  2  with val_accuracy: 0.9670  and test_accuracy: 0.9444
epoch  3  with loss: 0.1544  and accuracy: 0.9524
epoch  3  with val_accuracy: 0.9670  and test_accuracy: 0.9444
epoch  4  with loss: 0.1503  and accuracy: 0.9500
epoch  4  with val_accuracy: 0.9670  and test_accuracy: 0.9444
epoch  5  with loss: 0.1488  and accuracy: 0.9500
epoch  5  with val_accuracy: 0.9670  and test_accuracy: 0.9444
epoch  6  with loss: 0.1473  and accuracy: 0.9500
epoch  6  with val_accuracy: 0.9670  and test_accuracy: 0.9444
epoch  7  with loss: 0.1449  and accuracy: 0.9500
epoch  7  with val_accuracy: 0.9780  and test_accuracy: 0.9444
epoch  8  with loss: 0.1428  and accuracy: 0.9500
epoch  8  with val_accuracy: 0.9780  and test_

epoch  72  with loss: 0.0471  and accuracy: 0.9786
epoch  72  with val_accuracy: 0.9890  and test_accuracy: 0.9556


KeyboardInterrupt: 

In [ ]:
print(data_pred_labels[70])